In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json

In [2]:
html= requests.get('http://www.imdb.com/title/tt0043465/reviews?start=0')
soup = BeautifulSoup(html.text,"html.parser")

In [3]:
review = soup.find("div", {"id": "tn15content"})
review

<div id="tn15content">
<a class="btn large" href="/title/tt0043465/reviews-enter?ref_=tt_ur_urv" style="margin-top:10px;margin-bottom:10px">Write review</a><br/>
<style type="text/css"><!--
.click { cursor: pointer; cursor: hand; }
.yn { padding: 2px; background: #eeeeee; }
.yn form { display: inline; margin: 0; }
.yn input { font-weight: bold; padding: 0px 4px 8px; height: 20px }
.yn span { color: #008000; font-weight: bold; font-size: smaller; }
.yn span.error { color: #800000; }
.i1 { background: #eeffee; }
.i2 { background: #ffeeee; }
.i3 { background: #eeeeff; }
.i4 { background: #ffffee; }
.ucft { margin-bottom: 4px }
.ucft td { font-size: smaller }
.ucft .warning { color: #800000; font-weight: bold; }
.ucft input.save { padding: 0px; border: 0; background: none; color: #0000ff; text-decoration: underline; font-size: smaller; }
.ucft input.apply { font-weight: bold; padding: 0px 8px; height: 20px }
img.avatar {
    float: left;
    margin: 5px 10px 10px 0;
    border: 1px solid #

In [52]:
small_list

['46/52',
 '35/41',
 '39/50',
 '39/54',
 '32/47',
 '16/18',
 '17/21',
 '18/23',
 '13/16',
 '8/10']

In [49]:
reg = r"(<small>)(\d+)( out of )(\d+)(.+)"
re.sub(reg,'\\2/\\4',small_list[1])

'35/41'

In [51]:
for i in review.find_all('div'):
    for small in i.select_one('small'):
        small
        

TypeError: 'NoneType' object is not iterable

In [107]:
review.find_all('div')[6]

<div>
<small>39 out of 54 people found the following review useful:</small><br>
<a href="/user/ur0105696/"><img class="avatar" height="${avatar.image.size}" src="http://ia.media-imdb.com/images/M/MV5BMjI2NDEyMjYyMF5BMl5BanBnXkFtZTcwMzM3MDk0OQ@@._SX40_SY40_SS40_.jpg" width="${avatar.image.size}"/></a>
<h2>Can't stop watching it</h2>
<b>Author:</b>
<a href="/user/ur0105696/">ivan beshkov (ivanbeshkov@msn.com)</a> <small>from LOS ANGELES</small><br>
<small>9 February 2000</small><br>
</br></br></br></div>

In [99]:
f_user.a.get('href')

'/user/ur0307374/'

In [112]:
f_user = review.find_all('div')[0]

In [113]:
f_user.find('small').text

'46 out of 52 people found the following review useful:'

In [114]:
f_user.find('h2').text

'outstanding in its genre'

In [115]:
f_user.find_next_siblings('small')

[]

In [88]:
f_user.find('img',alt=True).get('alt')

'8/10'

In [80]:
class getReviewPerMovie(object):
    def __init__(self,movieNumber):
        self.site = 'http://www.imdb.com/title/'+str(movieNumber)+'/reviews'
        
    def getLength(self,categorie):
        first_page = self.site +'?filter='+ categorie + ';filter='+ categorie +';start='+'0'
        html_l = requests.get(first_page)
        soup_l = BeautifulSoup(html_l.text,"html.parser")   
        tbl = soup_l.findAll('table')[1]
        row_n = tbl.find_all('tr')
        str_len = row_n[0].find_all('td')[0].text
        regex = r"(Page).(\d).(of).(\d*)(.+)"
        len_review = re.sub(regex, '\\4',str_len)
        return(len_review)
    
    def postiveReviews(self):
        author = []
        title = []
        #comment_time = []
        rating = []
        usefulness = []
        for i in range(0,int(self.getLength('love'))):
            link= self.site +'?filter='+ 'love' + ';filter='+ 'love' +';start='+ str(i*10)
            print(link)
            html_link = requests.get(link)
            soup = BeautifulSoup(html_link.text,"html.parser")   
            review_soup = soup.find("div", {"id": "tn15content"})
            for div_re in review_soup.find_all('div'):
                rating.append(div_re.find('img',alt=True).get('alt'))
                author.append(re.sub(r"(/user/)(ur\d+)(/)",'\\2',div_re))
                title.append(div_re.h2.text)
                if div_re.small is not None:
                    print(div_re.small)
                    reg = r"(<small>)(\d+)( out of )(\d+)(.+)"
                    usefulness.append(re.sub(reg,'\\2/\\4',str(div_re.small)))
                
       


            
        



In [97]:
re.sub(r"(/user/)(ur\d+)(/)",'\\2','/user/ur0307374/')

'ur0307374'

In [76]:
any(i.isdigit() for i in '<small>from  898 UK</small>')

True

In [81]:
getReviewPerMovie('tt3783958').postiveReviews()

http://www.imdb.com/title/tt3783958/reviews?filter=love;filter=love;start=0
<small>321 out of 506 people found the following review useful:</small>
<small>260 out of 401 people found the following review useful:</small>
<small>184 out of 299 people found the following review useful:</small>
<small>168 out of 277 people found the following review useful:</small>
<small>174 out of 319 people found the following review useful:</small>
<small>105 out of 197 people found the following review useful:</small>
<small>103 out of 196 people found the following review useful:</small>
<small>70 out of 133 people found the following review useful:</small>
<small>11 out of 18 people found the following review useful:</small>
<small>7 out of 13 people found the following review useful:</small>
http://www.imdb.com/title/tt3783958/reviews?filter=love;filter=love;start=10
<small>from UK</small>
<small>from America</small>
<small>4 out of 8 people found the following review useful:</small>
<small>5 out o

['321/506',
 '260/401',
 '184/299',
 '168/277',
 '174/319',
 '105/197',
 '103/196',
 '70/133',
 '11/18',
 '7/13',
 '<small>from UK</small>',
 '<small>from America</small>',
 '4/8',
 '5/10',
 '7/14',
 '1/3',
 '2/5',
 '2/5',
 '2/5',
 '2/5',
 '2/5',
 '2/5',
 '2/5',
 '2/5',
 '2/5',
 '2/5',
 '3/7',
 '3/7',
 '3/7',
 '3/7',
 '3/7',
 '3/7',
 '3/7',
 '3/7',
 '3/7',
 '3/7',
 '3/7',
 '3/7',
 '3/7',
 '3/7',
 '4/9',
 '4/9',
 '4/9',
 '4/9',
 '4/9',
 '5/11',
 '5/11',
 '5/11',
 '5/11',
 '5/11',
 '6/13',
 '6/13',
 '7/15',
 '7/15',
 '0/2',
 '0/2',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '1/4',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '2/6',
 '3/8',
 '3/8',
 '3/8',
 '3/8',
 '3/8',
 '3/8',
 '3/8',
 '3/8',
 '4/10',
 '4/10',
 '4/10',
 '5/12',
 '5/12',
 '5/12',
 '5/12',
 

In [34]:
'http://www.imdb.com/title/'+str('tt3783958')+'/reviews'+'?filter='+ 'love' + ';filter='+ 'love' +';start='+'0'

'http://www.imdb.com/title/tt3783958/reviews?filter=love;filter=love;start=0'

In [32]:
 [ i*10 for i in range(0,getReviewPerMovie('tt3783958').postiveReviews())]

[0,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230,
 240,
 250,
 260,
 270,
 280,
 290,
 300,
 310,
 320,
 330,
 340]

In [21]:
getReviewPerMovie('tt3783958').postiveReviews()

35

In [ ]:
            
'''
    def PositiveReviews(self):
    

        
        
        find_len= 
        http://www.imdb.com/title/tt3783958/reviews?filter=love;filter=love;start=10
        http://www.imdb.com/title/tt3783958/reviews?filter=love;filter=love;start=0
        
'''       

In [4]:
num=str(10)
'http://www.imdb.com/title/tt3783958/reviews'+'?filter=love;filter=love;'+'start='+num

'http://www.imdb.com/title/tt3783958/reviews?filter=love;filter=love;start=10'

In [32]:
getUserReviewOnePage('http://www.imdb.com/title/tt0043465/reviews').soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<head><link href="http://adapi.ragapa.com/v1/rgp-location-layout-css?id=ucopia-prem1-050316" rel="stylesheet" type="text/css">
<meta content="text/html; charset=utf-8" http-equiv="content-type">
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime()};</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
var addClickstreamHeadersToAjax = function(xhr) {
    xhr.setRequestHeader("x-imdb-parent-id", "1G90HDGBM6XPNG4B1R5Q");
};
</script>
<title>Detective Story  Reviews &amp; Ratings - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<link href="http://www.imdb.com/title/tt0043465/reviews" rel="canonical"/><meta content="http://www.imdb.com/title/tt0043465/reviews" property="og:url"/>


In [ ]:
'http://www.imdb.com/title/tt0043465/reviews?start=60'